# Investigating the Indian startup ecosystem

## Business Understanding

### Description
The objective of this project is to analyse the indian start-up investment data over the course of four years (2018-2021).

#### Data Sources
- 2 SQL data sources (2021 & 2020)
- 2 CSV files (2019 & 2018)

### Null Hypothesis 
Investment amounts received by start-ups have no relation to sectors they operate in.


### Alternate Hypothesis
There is a relationship between investment amounts received by start-ups and the sectors they operate in

### Analytical Questions
1. Does location affect the amount of funding or investments?
2. Does the sector of start up affect the fundings?
3. How many companies need funding and are at what level of funding ?
4. Which stages give out the highest investment amounts?
5. Which cities have the highest number of startups and at what levels?
6. What are the levels of funding the startups are receiving per year?

In [1]:
#Libraries imported
import sqlalchemy as sa
import pyodbc  
from scipy import stats   
from scipy.stats import kruskal
from dotenv import dotenv_values 
import pandas as pd
import seaborn as sns
import warnings 


ModuleNotFoundError: No module named 'sqlalchemy'

In [ ]:
env_variables= dotenv_values('logins.env')
database= env_variables.get('database')
server = env_variables.get('server')
username = env_variables.get('username')
password = env_variables.get('password')



## Data Understanding



#### Connecting to the dapDB to extract the 2020 and 2021 data

In [ ]:
#Connecting to the database to analyse the 2020-2021 data

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"
connection = pyodbc.connect(connection_string)

In [ ]:
#query the 2020 startup funding data

query = "SELECT * FROM LP1_startup_funding2020"

data_2020 = pd.read_sql(query, connection)
data_2020.columns

In [ ]:
#Including the investment year to the data
data_2020['Investment_year'] = '2020'
data_2020=data_2020.reset_index()

In [ ]:
#Checking the datatypes of the columns
datatypes = data_2020.dtypes
datatypes

In [ ]:
#query the 2021 startup funding data
query = "SELECT * FROM LP1_startup_funding2021"

data_2021 = pd.read_sql(query, connection)
data_2021.head(5)


In [ ]:
#Checking the datatypes of the columns
datatypes = data_2021.dtypes
datatypes

In [ ]:
#Currency Converter function
def curr_converter(df,rate):
    amount_new=[]
    for a in df:
        if a.startswith('$'):
            
            amount_new.append(a.split('$')[1].replace(',',''))
        elif a.startswith('₹'): 
            
            amount_new.append(float((a.split('₹')[1]).replace(',',''))/rate )  
        else :
            amount_new.append(a)    
    return amount_new

In [ ]:
#Removing the dollar sign from data
data_2021[data_2021['Amount'].str.contains('$', na = False)]
data_2021['Amount']=data_2021.Amount.str.replace('\W', '', regex=True)


In [ ]:
data_2021['Founded'] = data_2021['Founded'].astype(str)
data_2021.head(2)

#### Realised that 2021 data had data entry errors whereby there was a mixup in inputing the data for the Investor,Amount and then the Stage columns

In [ ]:
#combining the Stage and Amount columns since there are mixed data entry errors
data_2021['new'] =  data_2021['Investor'].fillna('inv') +data_2021['Amount'].astype(str) + data_2021['Stage'].fillna('ab') 



In [ ]:
#Extract the amounts in the combined column to get the new amount column
data_2021['Amount_new']=data_2021['new'].str.extract('(\d+)')
data_2021.head(1)


In [ ]:

data_2021.drop(['new','Amount'], axis =1,inplace=True)
data_2021['Investment_year'] = '2021'

In [ ]:
#Renaming the Amount_new column
data_2021.rename(columns={'Amount_new':'Amount'}, inplace=True)
data_2021=data_2021.reset_index()

#2021 data cleaned!!!!!!!!!!!

In [ ]:
data_2021.head(1)

In [ ]:
#Reading 2018 data from the csv files

data_2018=pd.read_csv('startup_funding_2018_2019\startup_funding2018.csv')
data_2018.head(5)

In [ ]:
#Renaming column names of 2018 data to match all other datasets
data_2018.rename(columns={'Company Name':'Company_Brand','Industry':'Sector', 'Round/Series':'Stage', 'Location':'HeadQuarter', 'About Company':'What_it_does'}, inplace=True)
data_2018.head(5)

In [ ]:
#Removing the Lahk symbol and converting the Rupees to dollars
exch_rate= 158.38
amount_list=data_2018.Amount.tolist()
amount_new=curr_converter(amount_list,exch_rate)
data_2018['Amount']=amount_new
data_2018['Investment_year'] = '2018'
data_2018=data_2018.reset_index()

In [ ]:
#Reading 2019 data from the csv files
data_2019=pd.read_csv('startup_funding_2018_2019\startup_funding2019.csv')
data_2019['Founded'] = data_2019['Founded'].astype(str)
data_2019.head(5)

In [ ]:
#Renaming column names of 2019 data to match all other datasets
data_2019.rename(columns={'Company/Brand':'Company_Brand', 'What it does':'What_it_does', 'Amount($)':'Amount'}, inplace=True)
data_2019.columns


In [ ]:
#Removing the Lahk symbol and converting the Rupees to dollars
exch_rate= 177.13
amount_list=data_2019.Amount.tolist()
amount_new=curr_converter(amount_list,exch_rate)
data_2019['Amount']=amount_new
data_2019['Investment_year'] = '2019'
#data_2019=data_2019.reset_index()

### Merging datasets

In [ ]:
#Concating the four datasets since they have a similar structure to one big data
pd.set_option('display.max_rows', None)
final_df = pd.concat([data_2021,data_2020,data_2019,data_2018],axis=0,ignore_index=True)
final_df.head(5)


In [ ]:
#Saving the combined dataset to xlsx
final_df.to_csv("startup_funding_2018_2019\combined.csv",index=False
             ) 

# Data Cleaning & Exploration

In [ ]:
df= final_df
df.info()
df.head(2)

In [ ]:
df.shape
df['HeadQuarter'].unique()

In [ ]:
#split HQ column into two columns
df[['Town', 'City','Country']] = df['HeadQuarter'].str.split(',', expand=True)
df.head(2)


In [ ]:
# Function to fill new column based on conditions
def fill_new_column(row):
    if row['Town'] != '':
        return row['Town']
    else:
        return row['City']
    
# Create a new column filled with values from one column or default value if empty
df['Headquarter_City'] = df.apply(fill_new_column, axis=1)    

df.head(2)

In [ ]:
#Categorizing the startups into 9 groups
hq = df.Headquarter_City.tolist()
hq_cat=[]
hq_cat = ['Ahmedabad' if 'ahmedabad' in str(a).lower()  or 'ahmadabad' in str(a).lower()
         else 'Bangalore' if 'bangalore' in str(a).lower() 
         else 'Uttar Pradesh' if 'pradesh' in str(a).lower() 
         else 'Gurugram' if 'gurugram' in str(a).lower() 
         else 'Bangalore' if 'bang' in str(a).lower() 
         else 'Hyderabad' if 'hyderebad' in str(a).lower() or 'hyderabad' in str(a).lower()
         else a
         for a in hq]
hq_cat
df['Headquarter_City']=hq_cat

#### Categorising the Sectors into 9 groups for easier classification
- Technology & Software: Startups primarily focused on developing software, AI, IT solutions, and technology-related services.
- E-commerce & Retail: Startups involved in online retail, e-commerce platforms, marketplace solutions, and retail-focused businesses.
- Finance & FinTech: Startups operating in financial services, banking, financial technology (FinTech), cryptocurrency, and related areas.
- Healthcare & HealthTech: Startups in the healthcare industry, including health technology (HealthTech), telemedicine, medical devices, and healthcare services.
- Education: Startups in the Education industry, including learning,spacetech,data story telling and edtech.
- Energy: Startups in the Energy industry, including Renewable energy,battery technology, EV systems, crude oil and other energy related services.
- Agriculture: Startups in the Agriclture industry, including farming and fishing.
- Hospitality: Startups in the Hospitality industry, including real estate,hotels,homes and hospitality services.
- Others: Startups that do not fit directly into the above categories or have unique business models.


In [ ]:
#Categorizing the startups into 9 groups
sector = df.Sector.tolist()
cat=[]
cat = ['Finance & FinTech' if 'bank' in str(a).lower() or 'trading' in str(a).lower() or 'fintech' in str(a).lower() or 'vent' in str(a).lower() or 'capital' in str(a).lower() or 'insure' in str(a).lower() or 'crypt' in str(a).lower() or 'nft' in str(a).lower() or 'equity' in str(a).lower()
         else 'Technology & Software' if 'ai startup' in str(a).lower() or 'it' in str(a).lower() or 'chain' in str(a).lower() or 'augment' in str(a).lower() or 'robot' in str(a).lower() or 'gamin' in str(a).lower() or 'information' in str(a).lower() or 'mobile' in str(a).lower() or 'mlops' in str(a).lower() or 'biotech' in str(a).lower() or 'space' in str(a).lower() or 'nano' in str(a).lower()
         else 'E-commerce & Retail' if 'b2b' in str(a).lower() or 'e-com' in str(a).lower() or 'reta' in str(a).lower() or 'soci' in str(a).lower() or 'food' in str(a).lower() or 'content' in str(a).lower() or 'decor' in str(a).lower() or 'electronics' in str(a).lower() or 'apparel' in str(a).lower() or 'fashion' in str(a).lower() or 'food' in str(a).lower() or  'eye' in str(a).lower() or 'merchand' in str(a).lower()
         else 'Education' if 'edtech' in str(a).lower() or 'learn' in str(a).lower() or 'working' in str(a).lower() or 'story' in str(a).lower()  or 'edcation' in str(a).lower() or 'engineering' in str(a).lower()
         else 'Healthcare & HealthTech' if 'heal' in str(a).lower() or 'medi' in str(a).lower() or 'care' in str(a).lower() or 'hael' in str(a).lower() or 'nutri' in str(a).lower()
         else 'Agriculture' if 'farm' in str(a).lower() or 'fish' in str(a).lower() or 'annabi' in str(a).lower() or 'pollution' in str(a).lower()
         else 'Energy' if 'energy' in str(a).lower() or 'petro' in str(a).lower() or 'crude' in str(a).lower() or 'batter' in str(a).lower() or 'ev start' in str(a).lower() or 'solar' in str(a).lower() or 'ev' in str(a).lower()
         else 'Hospitality' if 'hospitalit' in str(a).lower() or 'mortg' in str(a).lower() or 'estate' in str(a).lower() or 'touri' in str(a).lower()
         else 'Other'
         for a in sector]


#### Categorizing the Stages
Categorizing the Stages into the 8 known groups of startup funding. Though there are other ways of funding such as debt financing ,IPO, etc there are eight (8) known groups of startup funding which are the seed funding and the other seven (7) in series from A to G.

In [ ]:
#Categorizing the Stages into the 9 known groups of startup funding 
stages = df.Stage.tolist()
stage=[]
stage = ['Series A' if 'series a' in str(a).lower() 
         else 'Series B' if 'series b' in str(a).lower() 
         else 'Series C' if 'series c' in str(a).lower()
         else 'Series D' if 'series d' in str(a).lower() 
         else 'Series E' if 'series e' in str(a).lower() 
         else 'Series F' if 'series f' in str(a).lower() 
         else 'Series G' if 'series g' in str(a).lower() 
         else 'Seed' if 'seed' in str(a).lower() 
         else 'Other'
         for a in stages]

In [ ]:
#Adding the category and categorised stages to the dataset
df['Category']=cat
df['Categorised_stage']=stage

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
#Dropping the Town,Other,City and column10 columns 
df.drop(['index','Town','Country','City','column10'],axis =1,inplace=True)

In [ ]:
#Converting the Amount column to numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')


In [ ]:
df.Category.unique()
df.head(1)

#### Filling the empty Amount rows
- Since investments made for startups can not be zero, we are replacing the 'nan' with the median investment of the sector that the startup finds itself in.
- Since investments made for startups are not less than $1000, we are replacing these rows with the average investment of the sector that the startup finds itself in.

In [ ]:
# Finding the medians of the respective category
cat_median_dict={}
for cat in df.Category.unique():
    cat_median_dict[cat] = df[df.Category == cat].Amount.median()

cat_median_dict

In [ ]:
# Finding the averages of the respective category
cat_average_dict={}
for cat in df.Category.unique():
    cat_average_dict[cat] = df[df.Category == cat].Amount.mean()

cat_average_dict

In [ ]:
#Filling the Amount values less than 1000 with their respective averages of their respective category
index_list = df.query("Amount < 1000").index.tolist()
for nums in index_list:
    cat=df.iloc[nums].Category
    old=df.iloc[nums].Amount
    df.replace(old,cat_average_dict[cat],inplace=True)

In [ ]:
#Fill nan with their respective medians of their respective category
index_list = df.query("Amount == 'nan'").index.tolist()
for nums in index_list:
    cat=df.iloc[nums].Category
    old=df.iloc[nums].Amount
    df.replace(old,cat_median_dict[cat],inplace=True)


In [ ]:
# Find nulls using .isnull()
df=df.replace(r'^s*$', float('NaN'), regex = True)

In [ ]:
# Sum of nulls using .isna()
df.isna().sum()

In [ ]:
df.dropna(subset = ['Amount'], inplace = True) 

In [ ]:
#finding duplicates
duplicate = df[df.duplicated()]
duplicate.shape

In [ ]:
#Converting the Amount column to numeric
df.describe().T

#### OBSERVATION
- There are no duplicates in the dataset

In [ ]:
### Only india cities

indian_cities=['Bangalore', 'Mumbai','Gurugram', 'New Delhi','Hyderabad','Jaipur', 'Ahmedabad', 'Chennai','Goa','Rajsamand','Ranchi','Faridabad', 'Gujarat','Thane','Pune','Cochin',
 'Noida', 'Chandigarh','Gurgaon','Vadodara','Kolkata', 'Haryana','Indore','Powai','Ghaziabad','Nagpur','West Bengal','Patna','Samsitpur','Lucknow','Telangana','Silvassa', 'Ambernath',
 'Panchkula','Surat', 'Andheri','Telugana','Bhubaneswar','Kottayam', 'Panaji','Coimbatore','Satara',
 'Orissia','Jodhpur','Santra','Trivandrum','Bhilwara','Kochi', 'The Nilgiris','Gandhinagar', 'Belgaum','Tirunelveli','Delhi', 'Dhingsara',
 'Bangaldesh','Samastipur','Rajastan','Ludhiana','Dehradun',
 'Kanpur','Odisha','Bihar','Uttar Pradesh','Banglore', 'Rajasthan','Bengaluru','Kalkaji','India',
 'Kota','Margão','Kannur','Kormangala','Mylapore','Guntur','Kalpakkam','Bhopal','Alleppey','Guindy']



df=df[df['Headquarter_City'].isin(indian_cities)]


In [ ]:
#Checking Normality of the data 

def check_normality(data,name):
    test_stat_normality, p_value_normality=stats.shapiro(data)
    print("p value:%.10f" % p_value_normality)
    if p_value_normality <0.05:
        print(f"Reject null hypothesis >> The data for {name} is not normally distributed")
    else:
        print(f"Fail to reject null hypothesis >> The data for {name} is normally distributed")

In [ ]:
fintech=df[df.Category=='Finance & FinTech']
tech = df[df.Category=='Technology & Software']
commerce=df[df.Category=='E-commerce & Retail']
health=df[df.Category=='Healthcare & HealthTech']
edu=df[df.Category=='Education']
agric = df[df.Category=='Agriculture']
energy = df[df.Category == 'Energy']
other=df[df.Category=='Other']


In [ ]:
fintech
check_normality(fintech.Amount,'Finance & FinTech')
check_normality(tech.Amount,'Technology & Software')
check_normality(commerce.Amount,'E-commerce & Retail')
check_normality(health.Amount,'Healthcare & HealthTech')
check_normality(edu.Amount,'Education')
check_normality(agric.Amount,'Agriculture')
check_normality(energy.Amount,'Energy')
check_normality(other.Amount,'Other')


In [ ]:
#Use an ANOVA test (READ ABOUT IT) (Non parametric tests)
#Using the P-Levene to test the Hypothesis
stat, pvalue_levene= stats.levene(fintech.Amount, commerce.Amount, health.Amount,tech.Amount,edu.Amount,agric.Amount,energy.Amount,other.Amount)

print("p value:%.4f" % pvalue_levene)
if pvalue_levene <0.05:
    print("Reject null hypothesis >> The variances of the samples are different.")
else:
    print("Fail to reject null hypothesis >> The variances of the samples are same.")

#### OBSERVATION
- Though the Variances of the samples are the same satisifying the homeostadasicity,the distribution of the samples differ therefore using ANOVA to test the hypothesis will introduce a significant level of error 
- Conducting a non-parametric test will be ideal (the Kruskal-Wallis test)

In [ ]:

stat, p = kruskal(fintech.Amount, commerce.Amount, health.Amount,tech.Amount,edu.Amount,agric.Amount,energy.Amount,other.Amount)
print('Statistics=%.3f, p=%.10f' % (stat, p))

if p > 0.05:
 print('All sample distributions are the same (fail to reject H0)')
else:
 print('One or more sample distributions are not equal distributions (reject null Hypothesis)')

### Analytical Questions

In [ ]:
df.head(2)

#### 1. Does location affect the amount of funding or investments?

In [ ]:
#Converting Amount to millions of dollars
df['Amount']=df.Amount/1000000
pd.options.display.float_format = '${:,.2f}'.format


In [ ]:
startup_loc= df.groupby('Headquarter_City')['Amount'].mean().head(10).sort_values(ascending=False)

startup_loc

In [ ]:
startup_loc.plot(kind="bar",title='Top Ten Average Investments per Headquarter Locations', legend=True,rot=30,y='Millions of Dollars')


#### 2. What is the minimum funding needed per Sector/Category of start up affect the fundings?

In [ ]:
startup_cat= df.groupby('Category')['Amount'].mean().sort_values(ascending=False)
startup_cat

In [ ]:
startup_cat.plot(kind="bar",title='Minimum Investments recorded per Category', legend=True,rot=30,y='Millions of Dollars')

#### 3. How many companies are in the different sectors of funding ?

In [ ]:
startup_stages= df.groupby(['Categorised_stage'])['Category'].count().sort_values(ascending=False)
startup_stages

In [ ]:
startup_stages.plot(kind="bar",title='Number of companies in the different funding sectors', legend=True,rot=30,y='Millions of Dollars')

#### 4. Which stages give out the highest investment amounts?

In [ ]:
startup_sectors= df.groupby('Categorised_stage')['Amount'].mean()
startup_sectors

In [ ]:
startup_sectors.plot(kind="bar",title='Number of companies in the different funding sectors', legend=True,rot=30,y='Millions of Dollars')

#### 5. Which cities have the highest number of startups and at what levels?

In [ ]:
startup_cities= df.groupby('Headquarter_City')['Categorised_stage'].count().head(5).sort_values(ascending=False)
startup_cities

In [ ]:
startup_cities.plot(kind="bar",title='Number of companies in the different funding sectors', legend=True,rot=30,y='Millions of Dollars')

#### 6. What are the levels of funding the startups are receiving per year?

In [ ]:
startup_year= df.groupby('Investment_year')['Amount'].mean().sort_values(ascending=True)
startup_year

In [ ]:
startup_year.plot(kind="bar",title='Number of companies in the different funding sectors', legend=True,rot=30,y='Millions of Dollars')